In [1]:
# 为这个项目导入需要的库
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # 允许为DataFrame使用display()
import matplotlib.pyplot as plt
import xgboost as xgb
import time

In [2]:
store = pd.read_csv("./rossmann-store-sales/store.csv")
sample = pd.read_csv("./rossmann-store-sales/sample_submission.csv")
test= pd.read_csv("./rossmann-store-sales/test.csv")
train = pd.read_csv("./rossmann-store-sales/train.csv")

C:\Users\wangk\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
store=store.fillna({'Promo2SinceWeek':0,'Promo2SinceYear':0,'CompetitionDistance':0,'PromoInterval':0})
store['CompetitionOpenSinceMonth']=store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].median())
store['CompetitionOpenSinceYear']=store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].median())

In [4]:
train['Sales'] = train['Sales'].apply(lambda x: np.log(x + 1))
store['CompetitionDistance'] = store['CompetitionDistance'].apply(lambda x: np.log(x + 1))
train['Customers'] = train['Customers'].apply(lambda x: np.log(x + 1))

In [5]:
 test_new=pd.merge(test,store,on='Store')

In [6]:
mappings={'a':1,'b':2,'c':3,'d':4}
test_new.StoreType.replace(mappings,inplace=True)
test_new.Assortment.replace(mappings,inplace=True)
test_new.StateHoliday.replace(mappings,inplace=True)
# inplace 表示就地修改

In [13]:
print(isinstance(test_new['StoreType'][1],np.int64))

True


In [16]:
test_new['Year']=test_new.Date.apply(lambda x:x.split('-')[0])
test_new['Year']=test_new['Year'].astype(float)
test_new['Month']=test_new.Date.apply(lambda x:x.split('-')[1])
test_new['Month']=test_new['Month'].astype(float)
test_new['Day']=test_new.Date.apply(lambda x:x.split('-')[2])
test_new['Day']=test_new['Day'].astype(float)

In [17]:
# 构建字典
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',7:'Jul', 8:'Aug',9:'Sept',10:'Oct',11:'Nov', 12:'Dec'}
test_new['monthStr'] = test_new.Month.map(month2str) # 将数字映射为月份
test_new.loc[test_new.PromoInterval == 0, 'PromoInterval'] = '' # 缺失值的字符串转换
test_new['IsPromoMonth'] = 0 # 创建一新列
for interval in test_new.PromoInterval.unique():
    if interval != '':
        for month in interval.split(','):
            test_new.loc[(test_new.monthStr == month) & (test_new.PromoInterval == interval), 'IsPromoMonth'] = 1


In [18]:
test_new['CompetitionOpen']=12*(test_new.Year-test_new.CompetitionOpenSinceYear)+(test_new.Month-test_new.CompetitionOpenSinceMonth)
test_new['CompetitionOpen']=test_new.CompetitionOpen.apply(lambda x:x if x>0 else 0)

In [19]:
'CompetitionOpen' in test_new.columns

True

In [16]:
test_new_final=test_new.drop(['Id','IsPromoMonth','Date','Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval','monthStr',
                              'CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Open'],axis=1)


In [17]:
train_new=pd.merge(train,store,on='Store')
train_new=train_new[(train_new['Open']==1)&(train_new['Sales']>0)&(train_new['Customers']>0)]
# 保证接受训练的数据都是有价值的

In [18]:
mappings={'a':1,'b':2,'c':3,'d':4}
train_new.StoreType.replace(mappings,inplace=True)
train_new.Assortment.replace(mappings,inplace=True)
train_new.StateHoliday.replace(mappings,inplace=True)

#Date 分成年月日
train_new['Year']=train_new.Date.apply(lambda x:x.split('-')[0])
train_new['Year']=train_new['Year'].astype(float)
train_new['Month']=train_new.Date.apply(lambda x:x.split('-')[1])
train_new['Month']=train_new['Month'].astype(float)
train_new['Day']=train_new.Date.apply(lambda x:x.split('-')[2])
train_new['Day']=train_new['Day'].astype(float)

#新增竞争者是否开业
train_new['CompetitionOpen']=12*(train_new.Year-train_new.CompetitionOpenSinceYear)+(train_new.Month-train_new.CompetitionOpenSinceMonth)
train_new['CompetitionOpen']=train_new.CompetitionOpen.apply(lambda x:x if x>0 else 0)


#判断train数据的当天是否有促销
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',7:'Jul', 8:'Aug',9:'Sept',10:'Oct',11:'Nov', 12:'Dec'}
train_new['monthStr'] = train_new.Month.map(month2str)
train_new.loc[train_new.PromoInterval == 0, 'PromoInterval'] = ''
train_new['IsPromoMonth'] = 0
for interval in train_new.PromoInterval.unique():
    if interval != '':
        for month in interval.split(','):
            train_new.loc[(train_new.monthStr == month) & (train_new.PromoInterval == interval), 'IsPromoMonth'] = 1
            
#去除掉重复的特征 
train_new_final=train_new.drop(['IsPromoMonth','Date','Customers','Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval','monthStr','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Open'],axis=1)
train_new_final=train_new_final.astype(float)
# train_new_final.head(20)

In [19]:
features=train_new_final.drop(['Sales'],axis=1)
Sales=train_new_final['Sales']



from sklearn.model_selection import train_test_split
# 将数据切分成训练集和测试集
x_train, x_test,y_train, y_test = train_test_split(features,Sales,test_size = 0.2,random_state = 0)


In [20]:

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

In [22]:

from xgboost.sklearn import XGBRegressor

Model_1 = XGBRegressor(learning_rate=0.1,n_estimators=400,max_depth=8)
Model_1.fit(x_train.values,y_train.values,eval_set = [(x_test.values,y_test.values)])
# eval_set 是验证集

y_pred = Model_1.predict(x_test.values)
error = rmspe(y_test,y_pred)
print("accuarcy: %.2f%%" % (error*100.0))


print("Make predictions on the test set")
Test_Pred = Model_1.predict(test_new_final.values)

result = pd.DataFrame({"Id": test_new['Id'], 'Sales': np.expm1(Test_Pred)})
# 由于之前我们对Sales数据进行了log（x+1）运算，所以这里要进行反运算
result.to_csv("Rossmann_submission_Model_1.csv", index=False)

[0]	validation_0-rmse:7.44374
[1]	validation_0-rmse:6.70122
[2]	validation_0-rmse:6.03314
[3]	validation_0-rmse:5.43204
[4]	validation_0-rmse:4.89128
[5]	validation_0-rmse:4.40484
[6]	validation_0-rmse:3.96732
[7]	validation_0-rmse:3.57379
[8]	validation_0-rmse:3.21990
[9]	validation_0-rmse:2.90176
[10]	validation_0-rmse:2.61582
[11]	validation_0-rmse:2.35883
[12]	validation_0-rmse:2.12802
[13]	validation_0-rmse:1.92071
[14]	validation_0-rmse:1.73470
[15]	validation_0-rmse:1.56800
[16]	validation_0-rmse:1.41838
[17]	validation_0-rmse:1.28469
[18]	validation_0-rmse:1.16508
[19]	validation_0-rmse:1.05806
[20]	validation_0-rmse:0.96281
[21]	validation_0-rmse:0.87763
[22]	validation_0-rmse:0.80207
[23]	validation_0-rmse:0.73532
[24]	validation_0-rmse:0.67614
[25]	validation_0-rmse:0.62340
[26]	validation_0-rmse:0.57793
[27]	validation_0-rmse:0.53770
[28]	validation_0-rmse:0.50305
[29]	validation_0-rmse:0.47280
[30]	validation_0-rmse:0.44714
[31]	validation_0-rmse:0.42414
[32]	validation_0-

[260]	validation_0-rmse:0.13750
[261]	validation_0-rmse:0.13735
[262]	validation_0-rmse:0.13707
[263]	validation_0-rmse:0.13684
[264]	validation_0-rmse:0.13674
[265]	validation_0-rmse:0.13657
[266]	validation_0-rmse:0.13624
[267]	validation_0-rmse:0.13598
[268]	validation_0-rmse:0.13567
[269]	validation_0-rmse:0.13554
[270]	validation_0-rmse:0.13526
[271]	validation_0-rmse:0.13488
[272]	validation_0-rmse:0.13463
[273]	validation_0-rmse:0.13444
[274]	validation_0-rmse:0.13416
[275]	validation_0-rmse:0.13391
[276]	validation_0-rmse:0.13377
[277]	validation_0-rmse:0.13367
[278]	validation_0-rmse:0.13353
[279]	validation_0-rmse:0.13333
[280]	validation_0-rmse:0.13311
[281]	validation_0-rmse:0.13300
[282]	validation_0-rmse:0.13269
[283]	validation_0-rmse:0.13258
[284]	validation_0-rmse:0.13246
[285]	validation_0-rmse:0.13233
[286]	validation_0-rmse:0.13208
[287]	validation_0-rmse:0.13203
[288]	validation_0-rmse:0.13186
[289]	validation_0-rmse:0.13161
[290]	validation_0-rmse:0.13157
[291]	va

In [24]:

Model_2 = XGBRegressor(learning_rate=0.1,n_estimators=100,max_depth=6,min_child_weight = 1,
                     subsample=0.8,colsample_btree=0.8,objective='reg:linear',
                     scale_pos_weight=1,random_state=27)

Model_2.fit(x_train.values,y_train.values,eval_set = [(x_test.values,y_test.values)],eval_metric = "rmse",
          early_stopping_rounds = 10)

### make prediction for test data
y_pred = Model_2.predict(x_test.values)
error = rmspe(y_test,y_pred)
print("accuarcy: %.2f%%" % (error*100.0))

[23:37:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:37:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_btree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:7.44386


C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-rmse:6.70143
[2]	validation_0-rmse:6.03344
[3]	validation_0-rmse:5.43247
[4]	validation_0-rmse:4.89181
[5]	validation_0-rmse:4.40544
[6]	validation_0-rmse:3.96805
[7]	validation_0-rmse:3.57463
[8]	validation_0-rmse:3.22088
[9]	validation_0-rmse:2.90292
[10]	validation_0-rmse:2.61717
[11]	validation_0-rmse:2.36047
[12]	validation_0-rmse:2.12990
[13]	validation_0-rmse:1.92296
[14]	validation_0-rmse:1.73734
[15]	validation_0-rmse:1.57097
[16]	validation_0-rmse:1.42198
[17]	validation_0-rmse:1.28859
[18]	validation_0-rmse:1.16913
[19]	validation_0-rmse:1.06277
[20]	validation_0-rmse:0.96812
[21]	validation_0-rmse:0.88391
[22]	validation_0-rmse:0.80912
[23]	validation_0-rmse:0.74316
[24]	validation_0-rmse:0.68482
[25]	validation_0-rmse:0.63349
[26]	validation_0-rmse:0.58860
[27]	validation_0-rmse:0.54933
[28]	validation_0-rmse:0.51566
[29]	validation_0-rmse:0.48625
[30]	validation_0-rmse:0.46123
[31]	validation_0-rmse:0.43936
[32]	validation_0-rmse:0.42065
[33]	validation_0

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'min_child_weight':[2,3,4],   
    'colsample_bytree':[0.5,0.7,1],
    'scale_pos_weight':[0.6,0.7,0.8]
}

xlf = XGBRegressor(learning_rate=0.1,n_estimators=50,max_depth=5,min_child_weight = 1,
                     subsample=0.8,colsample_btree=0.8,objective='reg:linear',
                     scale_pos_weight=1,random_state=27)
n_iter_search = 5

gsearch = RandomizedSearchCV(xlf,param_distributions=parameters,n_iter=n_iter_search, cv=2)



gsearch.fit(x_train.values,y_train.values,eval_set = [(x_test.values,y_test.values)],eval_metric = "rmse",
            early_stopping_rounds = 20)
print("Best score: %0.3f" % gsearch.best_score_)
best_estimator = gsearch.best_estimator_
print("Best parameters set",best_estimator)




[23:39:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:39:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_btree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:7.44389
[1]	validation_0-rmse:6.70151
[2]	validation_0-rmse:6.03356
[3]	validation_0-rmse:5.43281


C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-rmse:4.89212
[5]	validation_0-rmse:4.40604
[6]	validation_0-rmse:3.96870
[7]	validation_0-rmse:3.57553
[8]	validation_0-rmse:3.22186
[9]	validation_0-rmse:2.90418
[10]	validation_0-rmse:2.61843
[11]	validation_0-rmse:2.36202
[12]	validation_0-rmse:2.13160
[13]	validation_0-rmse:1.92470
[14]	validation_0-rmse:1.73937
[15]	validation_0-rmse:1.57303
[16]	validation_0-rmse:1.42417
[17]	validation_0-rmse:1.29109
[18]	validation_0-rmse:1.17219
[19]	validation_0-rmse:1.06597
[20]	validation_0-rmse:0.97154
[21]	validation_0-rmse:0.88745
[22]	validation_0-rmse:0.81307
[23]	validation_0-rmse:0.74738
[24]	validation_0-rmse:0.68930
[25]	validation_0-rmse:0.63862
[26]	validation_0-rmse:0.59434
[27]	validation_0-rmse:0.55556
[28]	validation_0-rmse:0.52190
[29]	validation_0-rmse:0.49297
[30]	validation_0-rmse:0.46838
[31]	validation_0-rmse:0.44729
[32]	validation_0-rmse:0.42878
[33]	validation_0-rmse:0.41369
[34]	validation_0-rmse:0.40093
[35]	validation_0-rmse:0.38916
[36]	validatio

C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\wangk\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-rmse:4.89216
[5]	validation_0-rmse:4.40607
[6]	validation_0-rmse:3.96867
[7]	validation_0-rmse:3.57554
[8]	validation_0-rmse:3.22186
[9]	validation_0-rmse:2.90419
[10]	validation_0-rmse:2.61847
[11]	validation_0-rmse:2.36204
[12]	validation_0-rmse:2.13155
[13]	validation_0-rmse:1.92470
[14]	validation_0-rmse:1.73926
[15]	validation_0-rmse:1.57301
[16]	validation_0-rmse:1.42404
[17]	validation_0-rmse:1.29092
[18]	validation_0-rmse:1.17192
[19]	validation_0-rmse:1.06565
[20]	validation_0-rmse:0.97118
[21]	validation_0-rmse:0.88698
[22]	validation_0-rmse:0.81269
[23]	validation_0-rmse:0.74693
[24]	validation_0-rmse:0.68864
[25]	validation_0-rmse:0.63782
[26]	validation_0-rmse:0.59324
[27]	validation_0-rmse:0.55463
[28]	validation_0-rmse:0.52091
[29]	validation_0-rmse:0.49127
